# 📊 Notebook 1: Data Collection

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pavannn16/BERTopic-arXiv-Analysis/blob/main/notebooks/01_data_collection.ipynb)

**Purpose:** Fetch 20,000 arXiv cs.AI paper abstracts using the arXiv API.

**Time:** ~15 minutes (API rate limited)

---

## 1. Setup and Installation

In [11]:
# Install required packages (run once in Colab)
!pip install arxiv pandas tqdm -q

In [12]:
# ============================================================
# PROJECT PATH SETUP - Works on Colab Web, VS Code, or Local
# ============================================================

import os
from pathlib import Path

# Detect environment and set project path
if 'google.colab' in str(get_ipython()):
    # Running on Google Colab - mount Drive
    from google.colab import drive
    drive.mount('/content/drive')
    PROJECT_PATH = '/content/drive/MyDrive/BERTopic-arXiv-Analysis'
    print("✅ Running on Google Colab")
else:
    # Running locally (VS Code, Jupyter, etc.)
    PROJECT_PATH = str(Path(os.getcwd()).parent) if 'notebooks' in os.getcwd() else os.getcwd()
    print("✅ Running locally")

# Create directory structure
for folder in ['data/raw', 'data/processed', 'data/embeddings', 'models', 'results/visualizations']:
    os.makedirs(f'{PROJECT_PATH}/{folder}', exist_ok=True)

print(f"📁 Project path: {PROJECT_PATH}")
print("📂 Directories ready:")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Running on Google Colab
📁 Project path: /content/drive/MyDrive/BERTopic-arXiv-Analysis
📂 Directories ready:


In [13]:
# Import libraries
import arxiv
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
import json
import time

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Configuration

In [14]:
# Configuration
CONFIG = {
    'category': 'cs.AI',           # arXiv category
    'max_results': 20000,          # Maximum papers to fetch (full scope)
    'months_back': 24,             # How many months of data (2 years)
    'batch_size': 100,             # Papers per API request
    'delay_seconds': 3.0,          # Delay between requests (be respectful)
}

# Calculate date range
end_date = datetime.now()
start_date = end_date - timedelta(days=CONFIG['months_back'] * 30)

print(f"Category: {CONFIG['category']}")
print(f"Date range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
print(f"Max results: {CONFIG['max_results']}")

Category: cs.AI
Date range: 2023-12-14 to 2025-12-03
Max results: 20000


## 3. Fetch Papers from arXiv API

In [15]:
def fetch_arxiv_papers(category, max_results, batch_size=100, delay=3.0):
    """
    Fetch papers from arXiv API.

    Args:
        category: arXiv category (e.g., 'cs.AI')
        max_results: Maximum number of papers
        batch_size: Papers per API request
        delay: Delay between requests in seconds

    Returns:
        List of paper dictionaries
    """
    query = f"cat:{category}"

    print(f"Fetching up to {max_results} papers from arXiv category: {category}")
    print(f"This may take {max_results * delay / 60 / batch_size:.1f} minutes...")

    # Configure search
    search = arxiv.Search(
        query=query,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.SubmittedDate,
        sort_order=arxiv.SortOrder.Descending
    )

    # Configure client
    client = arxiv.Client(
        page_size=batch_size,
        delay_seconds=delay,
        num_retries=5
    )

    papers = []

    try:
        for result in tqdm(client.results(search), total=max_results, desc="Fetching"):
            paper = {
                "arxiv_id": result.entry_id.split("/")[-1],
                "title": result.title.replace("\n", " ").strip(),
                "abstract": result.summary.replace("\n", " ").strip(),
                "authors": ", ".join([author.name for author in result.authors[:5]]),  # First 5 authors
                "date": result.published.strftime("%Y-%m-%d"),
                "year_month": result.published.strftime("%Y-%m"),
                "url": result.entry_id,
                "categories": ", ".join(result.categories),
                "primary_category": result.primary_category
            }
            papers.append(paper)

            if len(papers) >= max_results:
                break

    except Exception as e:
        print(f"\nError during fetch: {e}")
        print(f"Successfully fetched {len(papers)} papers before error")

    return papers

# Fetch papers
papers = fetch_arxiv_papers(
    category=CONFIG['category'],
    max_results=CONFIG['max_results'],
    batch_size=CONFIG['batch_size'],
    delay=CONFIG['delay_seconds']
)

print(f"\nTotal papers fetched: {len(papers)}")

Fetching up to 20000 papers from arXiv category: cs.AI
This may take 10.0 minutes...


Fetching:  50%|█████     | 10000/20000 [06:55<06:55, 24.09it/s]


Error during fetch: Page request resulted in HTTP 500 (https://export.arxiv.org/api/query?search_query=cat%3Acs.AI&id_list=&sortBy=submittedDate&sortOrder=descending&start=10000&max_results=100)
Successfully fetched 10000 papers before error

Total papers fetched: 10000


In [16]:
# arXiv API has a 10,000 result limit per query
# Let's fetch additional papers by querying different date ranges
# We already have 10,000 - let's get more from an earlier period

def fetch_arxiv_by_date_range(category, start_date, end_date, max_results=10000, batch_size=100, delay=3.0):
    """Fetch papers within a specific date range."""
    # Format dates for arXiv query
    start_str = start_date.strftime("%Y%m%d")
    end_str = end_date.strftime("%Y%m%d")

    query = f"cat:{category} AND submittedDate:[{start_str}0000 TO {end_str}2359]"

    print(f"Fetching papers from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")

    search = arxiv.Search(
        query=query,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.SubmittedDate,
        sort_order=arxiv.SortOrder.Descending
    )

    client = arxiv.Client(
        page_size=batch_size,
        delay_seconds=delay,
        num_retries=5
    )

    papers = []

    try:
        for result in tqdm(client.results(search), total=max_results, desc="Fetching"):
            paper = {
                "arxiv_id": result.entry_id.split("/")[-1],
                "title": result.title.replace("\n", " ").strip(),
                "abstract": result.summary.replace("\n", " ").strip(),
                "authors": ", ".join([author.name for author in result.authors[:5]]),
                "date": result.published.strftime("%Y-%m-%d"),
                "year_month": result.published.strftime("%Y-%m"),
                "url": result.entry_id,
                "categories": ", ".join(result.categories),
                "primary_category": result.primary_category
            }
            papers.append(paper)

            if len(papers) >= max_results:
                break

    except Exception as e:
        print(f"\nError: {e}")
        print(f"Fetched {len(papers)} papers before error")

    return papers

# Find the earliest date we have
earliest_date = pd.to_datetime(min([p['date'] for p in papers]))
print(f"Earliest paper in current batch: {earliest_date.strftime('%Y-%m-%d')}")

# Fetch earlier papers (before our current earliest date)
end_date_batch2 = earliest_date - timedelta(days=1)
start_date_batch2 = earliest_date - timedelta(days=180)  # 6 months earlier

print(f"\nFetching additional papers from {start_date_batch2.strftime('%Y-%m-%d')} to {end_date_batch2.strftime('%Y-%m-%d')}")

papers_batch2 = fetch_arxiv_by_date_range(
    category=CONFIG['category'],
    start_date=start_date_batch2,
    end_date=end_date_batch2,
    max_results=10000,
    batch_size=CONFIG['batch_size'],
    delay=CONFIG['delay_seconds']
)

print(f"\nAdditional papers fetched: {len(papers_batch2)}")

Earliest paper in current batch: 2025-09-26

Fetching additional papers from 2025-03-30 to 2025-09-25
Fetching papers from 2025-03-30 to 2025-09-25


Fetching: 100%|█████████▉| 9999/10000 [05:36<00:00, 29.70it/s]


Additional papers fetched: 10000


In [17]:
# Check what we have so far
print(f"Batch 1 (recent): {len(papers)} papers")
print(f"Batch 2 (earlier): {len(papers_batch2)} papers")

# Combine and deduplicate
all_papers = papers + papers_batch2
seen_ids = set()
unique_papers = []
for p in all_papers:
    if p['arxiv_id'] not in seen_ids:
        seen_ids.add(p['arxiv_id'])
        unique_papers.append(p)

print(f"Total unique papers: {len(unique_papers)}")

# Check date range
dates = [p['date'] for p in unique_papers]
print(f"Date range: {min(dates)} to {max(dates)}")

Batch 1 (recent): 10000 papers
Batch 2 (earlier): 10000 papers
Total unique papers: 20000
Date range: 2025-07-03 to 2025-12-02


In [18]:
# Use the combined unique papers
papers = unique_papers
print(f"✅ Final dataset: {len(papers)} papers")
print(f"📅 Date range: {min(dates)} to {max(dates)} (recent 5 months)")
print(f"🎯 Target achieved: 20,000 recent cs.AI papers!")

✅ Final dataset: 20000 papers
📅 Date range: 2025-07-03 to 2025-12-02 (recent 5 months)
🎯 Target achieved: 20,000 recent cs.AI papers!


## 4. Create DataFrame and Explore Data

In [19]:
# Create DataFrame
df = pd.DataFrame(papers)

print(f"DataFrame shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nDate range: {df['date'].min()} to {df['date'].max()}")
df.head()

DataFrame shape: (20000, 9)

Columns: ['arxiv_id', 'title', 'abstract', 'authors', 'date', 'year_month', 'url', 'categories', 'primary_category']

Date range: 2025-07-03 to 2025-12-02


,arxiv_id,title,abstract,authors,date,year_month,url,categories,primary_category
0,2512.02987v1,Fine-Tuned Large Language Models for Logical T...,Recent advances in natural language processing...,"Muyu Pan, Dheeraj Kodakandla, Mahfuza Farooque",2025-12-02,2025-12,http://arxiv.org/abs/2512.02987v1,"cs.CL, cs.AI",cs.CL
1,2512.02978v1,"Rethinking Generalized BCIs: Benchmarking 340,...",Robust decoding and classification of brain pa...,"Paul Barbaste, Olivier Oullier, Xavier Vasques",2025-12-02,2025-12,http://arxiv.org/abs/2512.02978v1,"q-bio.NC, cs.AI, cs.HC, cs.LG",q-bio.NC
2,2512.02966v1,Lumos: Let there be Language Model System Cert...,"We introduce the first principled framework, L...","Isha Chaudhary, Vedaant Jain, Avaljot Singh, K...",2025-12-02,2025-12,http://arxiv.org/abs/2512.02966v1,"cs.PL, cs.AI, cs.MA",cs.PL
3,2512.02942v1,Benchmarking Scientific Understanding and Reas...,The next frontier for video generation lies in...,"Lanxiang Hu, Abhilash Shankarampeta, Yixin Hua...",2025-12-02,2025-12,http://arxiv.org/abs/2512.02942v1,"cs.CV, cs.AI",cs.CV
4,2512.02932v1,EGGS: Exchangeable 2D/3D Gaussian Splatting fo...,Novel view synthesis (NVS) is crucial in compu...,"Yancheng Zhang, Guangyu Sun, Chen Chen",2025-12-02,2025-12,http://arxiv.org/abs/2512.02932v1,"cs.CV, cs.AI",cs.CV


In [20]:
# Basic statistics
print("Dataset Statistics:")
print(f"  Total papers: {len(df)}")
print(f"  Unique dates: {df['date'].nunique()}")
print(f"  Date range: {df['date'].min()} to {df['date'].max()}")

# Text length statistics
df['title_len'] = df['title'].str.len()
df['abstract_len'] = df['abstract'].str.len()

print(f"\nTitle length: mean={df['title_len'].mean():.0f}, median={df['title_len'].median():.0f}")
print(f"Abstract length: mean={df['abstract_len'].mean():.0f}, median={df['abstract_len'].median():.0f}")

Dataset Statistics:
  Total papers: 20000
  Unique dates: 153
  Date range: 2025-07-03 to 2025-12-02

Title length: mean=83, median=83
Abstract length: mean=1340, median=1340


In [21]:
# Papers per month
papers_per_month = df['year_month'].value_counts().sort_index()
print("Papers per month:")
print(papers_per_month)

Papers per month:
year_month
2025-07    3156
2025-08    3819
2025-09    4215
2025-10    4821
2025-11    3755
2025-12     234
Name: count, dtype: int64


In [22]:
# Sample abstracts
print("Sample abstracts:")
print("=" * 80)
for i, row in df.head(3).iterrows():
    print(f"\nTitle: {row['title']}")
    print(f"Date: {row['date']}")
    print(f"Abstract: {row['abstract'][:300]}...")
    print("-" * 80)

Sample abstracts:

Title: Fine-Tuned Large Language Models for Logical Translation: Reducing Hallucinations with Lang2Logic
Date: 2025-12-02
Abstract: Recent advances in natural language processing (NLP), particularly large language models (LLMs), have motivated the automatic translation of natural language statements into formal logic without human intervention. This enables automated reasoning and facilitates debugging, finding loop invariants, ...
--------------------------------------------------------------------------------

Title: Rethinking Generalized BCIs: Benchmarking 340,000+ Unique Algorithmic Configurations for EEG Mental Command Decoding
Date: 2025-12-02
Abstract: Robust decoding and classification of brain patterns measured with electroencephalography (EEG) remains a major challenge for real-world (i.e. outside scientific lab and medical facilities) brain-computer interface (BCI) applications due to well documented inter- and intra-participant variability. H...
--------

## 5. Save Raw Data

In [23]:
# Save raw data as JSON
raw_json_path = f"{PROJECT_PATH}/data/raw/arxiv_cs_ai_raw.json"
with open(raw_json_path, 'w') as f:
    json.dump(papers, f, indent=2)
print(f"Raw JSON saved to: {raw_json_path}")

# Save as CSV (more convenient for pandas)
raw_csv_path = f"{PROJECT_PATH}/data/raw/arxiv_cs_ai_raw.csv"
df.to_csv(raw_csv_path, index=False)
print(f"Raw CSV saved to: {raw_csv_path}")

print(f"\nTotal records saved: {len(df)}")

Raw JSON saved to: /content/drive/MyDrive/BERTopic-arXiv-Analysis/data/raw/arxiv_cs_ai_raw.json
Raw CSV saved to: /content/drive/MyDrive/BERTopic-arXiv-Analysis/data/raw/arxiv_cs_ai_raw.csv

Total records saved: 20000


## 6. Data Quality Checks

In [24]:
# Check for missing values
print("Missing values:")
print(df.isnull().sum())

# Check for duplicates
n_duplicates = df['arxiv_id'].duplicated().sum()
print(f"\nDuplicate arxiv_ids: {n_duplicates}")

# Check for empty abstracts
empty_abstracts = (df['abstract'].str.len() < 50).sum()
print(f"Short abstracts (<50 chars): {empty_abstracts}")

Missing values:
arxiv_id            0
title               0
abstract            0
authors             0
date                0
year_month          0
url                 0
categories          0
primary_category    0
title_len           0
abstract_len        0
dtype: int64

Duplicate arxiv_ids: 0
Short abstracts (<50 chars): 0


## Summary

This notebook has:
1. ✅ Fetched arXiv cs.AI papers using the API
2. ✅ Created a DataFrame with paper metadata
3. ✅ Performed initial data exploration
4. ✅ Saved raw data to Google Drive

**Next step:** Run `02_preprocessing.ipynb` to clean and prepare the text data.